In [15]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1
%aimport mermaid_generator

import os
import openai
import pandas as pd

with open("../API_KEY", "r") as f:
    key = f.read()

openai.api_key = key

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Make gpt mermaidify input

## Generate response

In [4]:
# system_input = """
# You are going to generate mermaid diagrams from the user input.
# You should exclusively output the mermaid code only (i.e. no text before or after so that the full response can be directly rendered as a mermaid diagram).

# The only exception to this is if the users input does not make sense to convert to a mermaid diagram, in which case you should start your message with "ERROR: " followed by a suggestion to the user to help them provide an input that could be converted to a mermaid diagram.
# """

system_input = """
You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
"""


def mermaidify(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input},
        ],
        temperature=0,
        top_p=1,
        n=1,
        # stop=["\n"],
        max_tokens=500,
    )
    return response


def display_mermaid_response(user_input, response):
    print(
        "\n".join(
            [
                f"System text:\n=====\n{system_input}\n======",
                f"User text:\n=====\n{user_input}\n======",
                f"Response text:\n=====\n{response.choices[0].message.content}\n======",
                f"Response end reason: {response.choices[0].finish_reason}",
                f"Prompt token usage: {response.usage.prompt_tokens}",
                f"Completion token usage: {response.usage.completion_tokens}",
                f"Total token usage: {response.usage.total_tokens}",
                f"Cost assuming gpt-3.5-turbo: {0.002/1000*response.usage.total_tokens*100:.4f} cents",
            ]
        )
    )

In [5]:
mermaid_user_input = "I'd like to represent some properties of a bunch of animals, for example, which have 4 legs, which have fur, which are birds/mammals/fish/etc"
good_response = mermaidify(mermaid_user_input)
display_mermaid_response(mermaid_user_input, good_response)

System text:
=====

You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.

User text:
=====
I'd like to represent some properties of a bunch of animals, for example, which have 4 legs, which have fur, which are birds/mammals/fish/etc
Response text:
=====
```mermaid
graph LR
A((Animals)) -->|4 legs| B(Mammals)
A -->|Feathers| C(Birds)
A -->|Scales| D(Fish)
B -->|Fur| E(Dogs)
B -->|Fur| F(Cats)
C -->|Beak| G(Eagles)
C -->|Beak| H(Owls)
D -->|Gills| I(Salmon)
D -->|Gills| J(Sharks)
```

Note: Thi

In [6]:
bad_user_input = "How tall is the eiffel tower?"
bad_response = mermaidify(bad_user_input)
display_mermaid_response(bad_user_input, bad_response)

System text:
=====

You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.

User text:
=====
How tall is the eiffel tower?
Response text:
=====
ERROR: This input cannot be translated into a mermaid diagram. Please provide a valid input that can be represented as a diagram.
Response end reason: stop
Prompt token usage: 134
Completion token usage: 26
Total token usage: 160
Cost assuming gpt-3.5-turbo: 0.0320 cents


## Put response into HTML file

In [7]:
import re


def extract_mermaid_code(text):
    """Extract the mermaid code part of a text response"""
    pattern = r"```mermaid(.+?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return None


def generate_html(mermaid_text):
    """Place the mermaid code into an html file"""
    content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <script type="module">
            import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
        </script>
        <script>mermaid.initialize({{startOnLoad:true, securityLevel: 'loose', theme: 'dark'}});</script>
    </head>
    <body>
        <pre class="mermaid"> 
            {mermaid_text}
        </pre>
    </body>
    </html lang="en">
    """
    with open("test.html", "w") as f:
        f.write(content)


mm_text = extract_mermaid_code(good_response.choices[0].message.content)
print(mm_text)
generate_html(mm_text)


graph LR
A((Animals)) -->|4 legs| B(Mammals)
A -->|Feathers| C(Birds)
A -->|Scales| D(Fish)
B -->|Fur| E(Dogs)
B -->|Fur| F(Cats)
C -->|Beak| G(Eagles)
C -->|Beak| H(Owls)
D -->|Gills| I(Salmon)
D -->|Gills| J(Sharks)



## Put it together into one

In [8]:
def mermaid_html_from_text(text):
    """Make a mermaid html file from natural language text input"""
    response = mermaidify(text)
    if response.choices[0].finish_reason != "stop":
        print(
            f"Response did not reach good stopping point, full response is\n\n{response}\n\n"
        )
    if "ERROR" in response.choices[0].message.content:
        print(
            f"Bad input text detected, response text = {response.choices[0].message.content}"
        )
        return False

    mm_text = extract_mermaid_code(response.choices[0].message.content)
    generate_html(mm_text)
    return True

In [9]:
mermaid_html_from_text(
    """
Can you write a mermaid graph that represents this code structure?

Class GameState:

Class PersistentState:

Class AgentState:

Class FriendlyAgentState(AgentState):

Class EnemyAgentState(EnemyState):

	
Class  Env:


Class Agent:


Class Plan:

Class FactoryPlan(Plan):

Class BalancedFactory(FactoryPlan):

Class MetalFactory(FactoryPlan):

Class WaterFactory(FactoryPlan):

Class PowerFactory(FactoryPlan):



Class ActionPlan(Plan):


Class CombatPlan(ActionPlan):


Class GuardResource(CombatPlan):


Class GuardLichen(CombatPlan):


Class Attack(CombatPlan):


Class Kamikazee(CombatPlan):


Class Waiting(ActionPlan):


Class SolarPanel(ActionPlan):


Class Mining(ActionPlan):


Class Clearing(ActionPlan):

Class RubbleClearing(Clearing):

Class LichenClearing(Clearing):


Class Transport(ActionPlan):
"""
)

True

## Turn it into a class that is easier to improve later

In [10]:
import re
import openai


class MermaidDiagramGenerator:
    def __init__(self):
        self.system_input = """
        You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
        """
        self.max_tokens = 2000

        self._mm_graph_types = [
            "graph",
            "sequenceDiagram",
            "gantt",
            "classDiagram",
            "gitGraph",
            "erDiagram",
            "journey",
        ]

        self._last_input = None
        self._last_response = None

    def get_mermaid_response(self, user_input):
        self._last_input = user_input
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": self.system_input},
                {"role": "user", "content": user_input},
            ],
            temperature=0,
            top_p=1,
            n=1,
            max_tokens=self.max_tokens,
        )
        self._last_response = response
        return response

    def display_response(self, response=None):
        response = self._last_response if response is None else response
        if response is None:
            print(f"No response to display")
            return
        print(
            "\n".join(
                [
                    f"System text:\n=====\n{self.system_input}\n======",
                    f"User text:\n=====\n{self._last_input}\n======",
                    f"Response text:\n=====\n{response.choices[0].message.content}\n======",
                    f"Response end reason: {response.choices[0].finish_reason}",
                    f"Prompt token usage: {response.usage.prompt_tokens}",
                    f"Completion token usage: {response.usage.completion_tokens}",
                    f"Total token usage: {response.usage.total_tokens}",
                    f"Cost assuming gpt-3.5-turbo: {0.002/1000*response.usage.total_tokens*100:.4f} cents",
                ]
            )
        )
        return

    def _extract_mermaid_code(self, text):
        """Extract the mermaid code part of a text response"""
        # If already formatted as only mm text
        if any([text.startswith(graph_type) for graph_type in self._mm_graph_types]):
            return text

        # If mm text inside code part of response
        if "```mermaid" in text:
            pattern = r"```mermaid(.+?)```"
            match = re.search(pattern, text, re.DOTALL)
        elif "```" in text:
            pattern = r"```(.+?)```"
            match = re.search(pattern, text, re.DOTALL)
        else:
            raise RuntimeError(f"Can't parse: \n {text}")

        if match:
            return match.group(1)
        else:
            raise RuntimeError(f"Can't parse: \n {text}")

    @staticmethod
    def _generate_html(mermaid_text, filename):
        """Place the mermaid code into an html file"""
        content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <script type="module">
                import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
            </script>
            <script>mermaid.initialize({{startOnLoad:true, securityLevel: 'loose', theme: 'dark', 
                flowchart: {{
                    useMaxWidth: true, // Disable maximum width for better zooming
                    htmlLabels: true, // Enable HTML-based labels for better styling
                    defaultRenderer: "elk", // Makes connections linear, ugly but good for large graphs
                }},
            }});
            </script>
        </head>
        <body>
            <pre class="mermaid"> 
                {mermaid_text}
            </pre>
        </body>
        </html lang="en">
        """
        with open(filename, "w") as f:
            f.write(content)

    def mermaidify(self, text, output_type="html", output_filename: str = "test.html"):
        """Make a mermaid graph from natural language text input
        Args:
            text: Any text to convert to mermaid graph
            output_type: 'html' to generate a file, or 'text' to get back the mermaid text only
            output_filename: if 'html' output type, then save the file in 'output_filename'

        Returns:
            mermaid graph
        """
        if output_type not in ["html", "text"]:
            raise ValueError(
                f'output_type must be one of ["html", "text"], got {output_type}'
            )

        response = self.get_mermaid_response(text)
        if response.choices[0].finish_reason != "stop":
            print(f"Response did not reach good stopping point")
            self.display_response(response)
        if "ERROR" in response.choices[0].message.content:
            print(
                f"Bad input text detected, response text = {response.choices[0].message.content}"
            )
            self.display_response(response)
            return None

        mm_text = self._extract_mermaid_code(response.choices[0].message.content)
        if output_type == "html":
            self._generate_html(mm_text, output_filename)
        return mm_text

In [11]:
mdg = MermaidDiagramGenerator()
mdg.mermaidify(
    """
    A goes to B, B goes to C, and D goes to A, B, and C
""",
    output_type="html",
    output_filename="short_test.html",
)
mdg.display_response()

System text:
=====

        You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
        
User text:
=====

    A goes to B, B goes to C, and D goes to A, B, and C

Response text:
=====
```mermaid
graph LR
A --> B
B --> C
D --> A
D --> B
D --> C
```
Response end reason: stop
Prompt token usage: 151
Completion token usage: 28
Total token usage: 179
Cost assuming gpt-3.5-turbo: 0.0358 cents


In [12]:
mdg.mermaidify(
    """
Can you write a mermaid graph that represents this code structure?

Class GameState:

Class PersistentState:

Class AgentState:

Class FriendlyAgentState(AgentState):

Class EnemyAgentState(EnemyState):


Class  Env:


Class Agent:


Class Plan:

Class FactoryPlan(Plan):

Class BalancedFactory(FactoryPlan):

Class MetalFactory(FactoryPlan):

Class WaterFactory(FactoryPlan):

Class PowerFactory(FactoryPlan):


Class ActionPlan(Plan):


Class CombatPlan(ActionPlan):


Class GuardResource(CombatPlan):


Class GuardLichen(CombatPlan):


Class Attack(CombatPlan):


Class Kamikazee(CombatPlan):


Class Waiting(ActionPlan):


Class SolarPanel(ActionPlan):


Class Mining(ActionPlan):


Class Clearing(ActionPlan):

Class RubbleClearing(Clearing):

Class LichenClearing(Clearing):


Class Transport(ActionPlan):
""",
    output_type="html",
    output_filename="long_test.html",
)
mdg.display_response()

System text:
=====

        You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
        
User text:
=====

Can you write a mermaid graph that represents this code structure?

Class GameState:

Class PersistentState:

Class AgentState:

Class FriendlyAgentState(AgentState):

Class EnemyAgentState(EnemyState):


Class  Env:


Class Agent:


Class Plan:

Class FactoryPlan(Plan):

Class BalancedFactory(FactoryPlan):

Class MetalFactory(FactoryPlan):

Class WaterFactory(FactoryPlan):

Cla

### NOTE: Test below fails due to trying to put multiple class diagrams in one html pre 

In [13]:
mdg.mermaidify(
    """
Can you modify this to be three separate diagrams, one for the classes related to GameState, one for the classes related to AgentState, and one for classes related to Plan

classDiagram
    Class GameState
    Class PersistentState
    Class AgentState
    Class FriendlyAgentState
    Class EnemyAgentState
    Class Env
    Class Agent
    Class Plan
    Class FactoryPlan
    Class BalancedFactory
    Class MetalFactory
    Class WaterFactory
    Class PowerFactory
    Class ActionPlan
    Class CombatPlan
    Class GuardResource
    Class GuardLichen
    Class Attack
    Class Kamikazee
    Class Waiting
    Class SolarPanel
    Class Mining
    Class Clearing
    Class RubbleClearing
    Class LichenClearing
    Class Transport

    GameState <|-- PersistentState
    AgentState <|-- FriendlyAgentState
    AgentState <|-- EnemyAgentState
    Plan <|-- FactoryPlan
    FactoryPlan <|-- BalancedFactory
    FactoryPlan <|-- MetalFactory
    FactoryPlan <|-- WaterFactory
    FactoryPlan <|-- PowerFactory
    Plan <|-- ActionPlan
    ActionPlan <|-- CombatPlan
    CombatPlan <|-- GuardResource
    CombatPlan <|-- GuardLichen
    CombatPlan <|-- Attack
    CombatPlan <|-- Kamikazee
    ActionPlan <|-- Waiting
    ActionPlan <|-- SolarPanel
    ActionPlan <|-- Mining
    ActionPlan <|-- Clearing
    Clearing <|-- RubbleClearing
    Clearing <|-- LichenClearing
    ActionPlan <|-- Transport
""",
    output_type="html",
    output_filename="modify_test.html",
)
mdg.display_response()

System text:
=====

        You are an AI specialized in generating mermaid diagrams from user input. Your task is to translate the user's description into a mermaid diagram code. Remember, your response should contain only the mermaid code - no explanations, no additional text or context. The code should be ready to be rendered as a mermaid diagram immediately.

        In case the user's input is unclear or cannot be translated into a mermaid diagram, start your response with "ERROR: ", followed by a suggestion to help them provide input that can be converted into a mermaid diagram.
        
User text:
=====

Can you modify this to be three separate diagrams, one for the classes related to GameState, one for the classes related to AgentState, and one for classes related to Plan

classDiagram
    Class GameState
    Class PersistentState
    Class AgentState
    Class FriendlyAgentState
    Class EnemyAgentState
    Class Env
    Class Agent
    Class Plan
    Class FactoryPlan
    Cl

In [ ]:
from mermaid_generator import 